In [ ]:
!pip install fosforml

In [1]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

In [2]:
file_name = 'ATTRITION_PREDICTIVE_DATA1'

In [3]:
sf_df = my_session.sql("select * from {}".format(file_name ))


In [4]:
import pandas as pd
pandas_df = sf_df.to_pandas()
#pandas_df = pd.read_csv(file_name)

In [5]:
print(pandas_df.isnull().sum())

EMPLOYEE_ID                      0
TENURE_MONTHS                    0
BIRTH_YEAR                       0
AGE                              0
SENIORITY                        0
SCHOOL_ENDDATE                   0
JOB_STARTDATE                    0
JOB_ENDDATE                  90076
CITY                             0
DISTANCE                         0
DEGREE_CLEAN                     0
ETHNICITY                        0
MARITAL_STATUS                   0
ROLE                             0
COMPANY_NAME                     0
ORGANIZATION_TYPE                0
ORGANIZATION_OWNERSHIP           0
STATE                            0
COUNTRY                          0
GENDER                           0
WORK_LIFE_BALANCE                0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
SALARY                           0
JOB_SATISFACTION                 0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
OVER_TIME                        0
RELATIONSHIP_SATISFA

In [6]:
pandas_df = pandas_df.dropna()

In [7]:
print(pandas_df.isnull().sum())

EMPLOYEE_ID                  0
TENURE_MONTHS                0
BIRTH_YEAR                   0
AGE                          0
SENIORITY                    0
SCHOOL_ENDDATE               0
JOB_STARTDATE                0
JOB_ENDDATE                  0
CITY                         0
DISTANCE                     0
DEGREE_CLEAN                 0
ETHNICITY                    0
MARITAL_STATUS               0
ROLE                         0
COMPANY_NAME                 0
ORGANIZATION_TYPE            0
ORGANIZATION_OWNERSHIP       0
STATE                        0
COUNTRY                      0
GENDER                       0
WORK_LIFE_BALANCE            0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
SALARY                       0
JOB_SATISFACTION             0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
OVER_TIME                    0
RELATIONSHIP_SATISFACTION    0
CHURN                        0
dtype: int64


In [8]:
#dataset = pandas_df[["DISTANCE","DEGREE_CLEAN","ROLE","COMPANY_NAME","ORGANIZATION_TYPE","ORGANIZATION_OWNERSHIP","JOB_ENDDATE","CHURN"]]
dataset = pandas_df[["ORGANIZATION_TYPE","JOB_ENDDATE","CHURN"]]

In [9]:
print(dataset)

         ORGANIZATION_TYPE JOB_ENDDATE  CHURN
0       Financial Services  2019-08-01      0
1                Aerospace  2021-06-01      1
3       Financial Services  2017-07-01      1
4               E-commerce  2019-08-01      0
6               E-commerce  2018-07-01      1
...                    ...         ...    ...
299995          Investment  2023-05-01      0
299996          Technology  2023-05-01      0
299997       Environmental  2023-05-01      0
299998          Technology  2023-05-01      0
299999       Environmental  2023-06-01      0

[209924 rows x 3 columns]


In [10]:
dataset['JOB_ENDDATE'] = pd.to_datetime(dataset['JOB_ENDDATE'],format= '%d-%m-%Y').dt.strftime('%d-%m-%Y')